In [1]:
# Reference:
# https://www.kaggle.com/demetrypascal/fork-of-2heads-looper-super-puper-plate/notebook

kernel_mode = True

# Preparations

Let’s load the packages and provide some constants for our script:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from tensorflow.keras import layers, regularizers, Sequential, Model, backend, callbacks, optimizers, metrics, losses
import tensorflow as tf
import sys
import os
import random
import json
sys.path.append('../input/iterative-stratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pickle
from pickle import dump, load
import glob

import warnings
warnings.filterwarnings('ignore')

In [3]:
PATH = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA"
model_output_folder = "../input/improving-mark-s-2-heads-model" if kernel_mode \
    else f"{PATH}/improving-mark-s-2-heads-model"
os.makedirs(model_output_folder, exist_ok=True)

# SEEDS = [23]
SEEDS = [23, 228, 1488, 1998, 2208, 2077, 404]
KFOLDS = 10

batch_size = 256
# batch_size = 128

label_smoothing_alpha = 0.0005

P_MIN = label_smoothing_alpha
P_MAX = 1 - P_MIN

In [4]:
# Import train data, drop sig_id, cp_type
train_features = pd.read_csv(f'{PATH}/train_features.csv')

non_ctl_idx = train_features.loc[
    train_features['cp_type'] != 'ctl_vehicle'].index.to_list()

# Drop training data with ctl vehicle
tr = train_features.iloc[non_ctl_idx, :].reset_index(drop=True)

test_features = pd.read_csv(f'{PATH}/test_features.csv')
te = test_features.copy()

In [5]:
train_targets_scored = pd.read_csv(f'{PATH}/train_targets_scored.csv')
Y = train_targets_scored.drop('sig_id', axis=1)
Y = Y.iloc[non_ctl_idx, :].copy().reset_index(drop=True).values

train_targets_nonscored = pd.read_csv(f'{PATH}/train_targets_nonscored.csv')
Y0 = train_targets_nonscored.drop('sig_id', axis=1)
Y0 = Y0.iloc[non_ctl_idx, :].copy().reset_index(drop=True).values

sub = pd.read_csv(f'{PATH}/sample_submission.csv')
sub.iloc[:, 1:] = 0

# Features from t.test

Here I am getting most important predictors

In [6]:
# Import predictors from public kernel
json_file_path = '../input/t-test-pca-rfe-logistic-regression/main_predictors.json' if kernel_mode \
    else "/workspace/Kaggle/MoA/t-test-pca-rfe-logistic-regression/main_predictors.json"

with open(json_file_path, 'r') as j:
    predictors = json.loads(j.read())
    predictors = predictors['start_predictors']

In [7]:
second_Xtrain = tr[predictors].copy().values

second_Xtest = te[predictors].copy().values
second_Xtrain.shape

(21948, 447)

# Keras model

I got idea of **label smoothing** from this notebook: https://www.kaggle.com/kailex/moa-transfer-recipe-with-smoothing

In [8]:
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, P_MIN, P_MAX)
    return -backend.mean(y_true * backend.log(y_pred) +
                         (1 - y_true) * backend.log(1 - y_pred))

# Inference

In [9]:
numeric_features = [c for c in train_features.columns if c != "sig_id"]
gene_experssion_features = [c for c in numeric_features if c.startswith("g-")]
cell_viability_features = [c for c in numeric_features if c.startswith("c-")]
len(gene_experssion_features), len(cell_viability_features)

(772, 100)

In [10]:
tr = pd.get_dummies(tr, columns=["cp_time", "cp_dose"])

In [11]:
tr = tr.drop(['sig_id', 'cp_type'], axis=1)
te = test_features.drop(['sig_id', 'cp_type'], axis=1)

In [12]:
te = pd.get_dummies(te, columns=["cp_time", "cp_dose"])

In [13]:
def preprocessor_1(test, seed, scaler=None, pca_gs=None, pca_cs=None):
    n_gs = 2
    n_cs = 100

    # g-mean, c-mean
    test_g_mean = test[gene_experssion_features].mean(axis=1)

    test_c_mean = test[cell_viability_features].mean(axis=1)

    test_columns = test.columns.tolist()

    test = np.concatenate(
        (test, test_g_mean[:, np.newaxis], test_c_mean[:, np.newaxis]), axis=1)

    # Standard Scaler for Numerical Values
    test = pd.DataFrame(data=scaler.transform(test),
                        columns=test_columns + ["g_mean", "c_mean"])
    test_pca_gs = pca_gs.transform(test[gene_experssion_features].values)

    test_pca_cs = pca_cs.transform(test[cell_viability_features].values)

    # Append Features
    test = np.concatenate((test, test_pca_gs, test_pca_cs), axis=1)

    return test


def preprocessor_2(test, scaler=None):
    # Standard Scaler for Numerical Values
    test = scaler.transform(test)

    return test, scaler


def save_pickle(obj, model_output_folder, name):
    dump(obj, open(f"{model_output_folder}/{name}.pkl", 'wb'),
         pickle.HIGHEST_PROTOCOL)


def load_pickle(model_output_folder, name):
    return load(open(f"{model_output_folder}/{name}.pkl", 'rb'))


def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.mean(-logloss)

In [14]:
tr.shape, te.shape

((21948, 877), (3982, 877))

In [15]:
oof_predictions = np.zeros((tr.shape[0], Y.shape[1]))

y_pred = np.zeros((te.shape[0], 206))
for s in SEEDS:

    random.seed(s)
    np.random.seed(s)
    tf.random.set_seed(s)

    k = 0
    kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=s)
    for train_index, valid_index in kf.split(tr):
        file_name = f"seed{s}_fold{k}"

        print(f"Inferencing on seed{s} fold{k} ......")

        scaler_1 = load_pickle(model_output_folder, f"{file_name}_scaler_1")
        pca_gs = load_pickle(model_output_folder, f"{file_name}_pca_gs")
        pca_cs = load_pickle(model_output_folder, f"{file_name}_pca_cs")
        X_test_1 = preprocessor_1(te, s, scaler_1, pca_gs, pca_cs)

        scaler_2 = load_pickle(model_output_folder, f"{file_name}_scaler_2")
        X_test_2, scaler_2 = preprocessor_2(second_Xtest, scaler_2)

        y_valid_1 = Y[valid_index, :]
        y_valid_2 = Y0[valid_index, :]

        n_features = X_test_1.shape[1]
        n_features_2 = X_test_2.shape[1]

        # Model Definition #

        input1_ = layers.Input(shape=(n_features, ))
        input2_ = layers.Input(shape=(n_features_2, ))

        output1 = Sequential([
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(512, activation="elu"),
            layers.BatchNormalization(),
            layers.Dense(256, activation="elu")
        ])(input1_)

        answer1 = Sequential([
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(512, "relu")
        ])(layers.Concatenate()([output1, input2_]))

        answer2 = Sequential([
            layers.BatchNormalization(),
            layers.Dense(512, "elu"),
            layers.BatchNormalization(),
            layers.Dense(256, "relu")
        ])(layers.Concatenate()([output1, input2_, answer1]))

        answer3 = Sequential(
            [layers.BatchNormalization(),
             layers.Dense(256,
                          "elu")])(layers.Concatenate()([answer1, answer2]))

        answer3_ = Sequential([
            layers.BatchNormalization(),
            layers.Dense(256, "relu")
        ])(layers.Concatenate()([answer1, answer2, answer3]))

        answer4 = Sequential([
            layers.BatchNormalization(),
            layers.Dense(
                256,
                kernel_initializer=tf.keras.initializers.lecun_normal(seed=s),
                activation='selu',
                name='last_frozen'),
            layers.BatchNormalization(),
            layers.Dense(
                206,
                kernel_initializer=tf.keras.initializers.lecun_normal(seed=s),
                activation='selu')
        ])(layers.Concatenate()([output1, answer2, answer3, answer3_]))

        # Scored Training #

        answer5 = Sequential(
            [layers.BatchNormalization(),
             layers.Dense(Y.shape[1], "sigmoid")])(answer4)

        m_nn = tf.keras.Model([input1_, input2_], answer5)

        m_nn.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                     loss=losses.BinaryCrossentropy(
                         label_smoothing=label_smoothing_alpha),
                     metrics=logloss)

        # Load final model
        m_nn = tf.keras.models.load_model(
            f'{model_output_folder}/{file_name}_final.h5',
            custom_objects={'logloss': logloss})

        # Generate Submission Prediction #
        fold_submit_preds = m_nn.predict([X_test_1, X_test_2],
                                         batch_size=batch_size)
        y_pred += fold_submit_preds / (KFOLDS * len(SEEDS))
        print(fold_submit_preds[:5, :])

        k += 1

        print('\n')

Inferencing on seed23 fold0 ......
[[0.00088883 0.00099666 0.00237347 ... 0.00199676 0.00215248 0.00156668]
 [0.00026446 0.00374281 0.00038004 ... 0.0012615  0.00035498 0.00553844]
 [0.00087794 0.00048877 0.00367082 ... 0.00603043 0.00936385 0.00946245]
 [0.00062814 0.00081086 0.00251164 ... 0.00126885 0.00725744 0.00635387]
 [0.00172297 0.00147587 0.00098024 ... 0.00151264 0.00023294 0.00041767]]


Inferencing on seed23 fold1 ......
[[0.00046394 0.0010355  0.00084425 ... 0.00282262 0.0023657  0.00192209]
 [0.00032254 0.00068681 0.0008272  ... 0.00330355 0.00420385 0.00211305]
 [0.00081506 0.00037118 0.00263101 ... 0.00217702 0.00610909 0.0033968 ]
 [0.0018273  0.00122982 0.00265053 ... 0.003655   0.00526577 0.00193695]
 [0.00077879 0.00093191 0.00129182 ... 0.00182818 0.00055576 0.00089056]]


Inferencing on seed23 fold2 ......
[[0.00099687 0.00200254 0.00582322 ... 0.00571227 0.00533306 0.0018903 ]
 [0.00018584 0.00052399 0.00066122 ... 0.00023952 0.00408332 0.00053148]
 [0.00048569 

[[2.34454812e-04 6.18035381e-04 1.26391987e-03 ... 1.23734737e-03
  1.16126714e-02 4.70787403e-04]
 [1.72041531e-04 1.12038004e-04 1.40206190e-04 ... 2.12331477e-04
  3.85334971e-03 6.46615122e-03]
 [6.70590525e-05 1.74687855e-04 2.80793849e-03 ... 1.41214998e-03
  5.56204468e-04 5.29715698e-03]
 [7.76369241e-04 5.18749526e-04 8.39014479e-04 ... 1.50220608e-03
  1.57309012e-04 1.05450663e-03]
 [4.98102745e-04 9.80538316e-04 2.97269411e-03 ... 1.86360406e-03
  1.92408287e-03 1.28407534e-02]]


Inferencing on seed1488 fold0 ......
[[0.00029114 0.00013722 0.0007797  ... 0.00190905 0.00137594 0.00126921]
 [0.00034279 0.00097791 0.00042131 ... 0.00018181 0.00135878 0.00177868]
 [0.00015142 0.00074122 0.00581798 ... 0.00836761 0.00080036 0.00295768]
 [0.00026584 0.0009495  0.00283485 ... 0.00250359 0.00062955 0.00673462]
 [0.00182102 0.0020759  0.00281806 ... 0.00075051 0.00021723 0.00032587]]


Inferencing on seed1488 fold1 ......
[[0.00045044 0.00100445 0.00117782 ... 0.00136825 0.00126553

[[0.00051531 0.00014334 0.00127858 ... 0.00300946 0.00562385 0.00357563]
 [0.00075596 0.00163559 0.00417796 ... 0.00101133 0.00376075 0.00073683]
 [0.00097048 0.0002729  0.00113416 ... 0.00099672 0.00972803 0.00134865]
 [0.00104264 0.00032659 0.00091217 ... 0.00096572 0.00169761 0.0031026 ]
 [0.00129145 0.00032251 0.00143046 ... 0.00269461 0.00064149 0.00171212]]


Inferencing on seed1998 fold8 ......
[[0.00042782 0.00048753 0.00178385 ... 0.00161457 0.00754036 0.00152253]
 [0.00013164 0.00046972 0.0009485  ... 0.00097202 0.00103994 0.00143752]
 [0.00103449 0.00032872 0.00334472 ... 0.00672495 0.00112143 0.00310236]
 [0.00059287 0.00032429 0.00345052 ... 0.00360266 0.00096988 0.00217621]
 [0.00165306 0.00174202 0.00195608 ... 0.00195153 0.00045524 0.00140959]]


Inferencing on seed1998 fold9 ......
[[1.2730838e-04 3.3339064e-04 2.0858780e-03 ... 1.5366664e-03
  3.2930935e-03 5.3673959e-03]
 [9.3060436e-05 3.1549647e-04 5.0779507e-03 ... 2.5134352e-03
  2.0834932e-02 6.2641976e-03]
 [7.

[[7.4588327e-04 1.8574045e-03 1.3377188e-03 ... 9.3964976e-04
  2.1188054e-03 1.6596019e-03]
 [5.8701407e-04 6.2859379e-04 1.6938129e-03 ... 1.2068810e-03
  8.0198655e-03 2.9389106e-03]
 [1.1586806e-04 9.5075236e-05 5.2451249e-03 ... 1.4030220e-03
  1.3890655e-03 2.3437778e-03]
 [3.5204587e-04 7.8275299e-04 1.7949380e-03 ... 1.4334045e-03
  7.6205714e-04 1.0678188e-03]
 [1.7040166e-03 3.1096069e-03 1.0554589e-03 ... 9.2603092e-04
  6.1205623e-04 7.2671846e-04]]


Inferencing on seed2077 fold7 ......
[[4.9968436e-04 3.1988358e-04 2.7461315e-03 ... 8.0828235e-04
  9.8520853e-03 2.4191560e-03]
 [4.0632128e-04 8.4553740e-04 8.3364837e-04 ... 1.4547567e-03
  1.2269866e-03 9.3095057e-04]
 [1.3041876e-04 6.6570545e-05 2.7293076e-03 ... 1.8883619e-03
  3.5894566e-04 1.7246323e-03]
 [5.2579754e-04 2.8514388e-04 4.3431078e-03 ... 3.5569596e-03
  3.2581883e-03 7.8947209e-03]
 [1.8055832e-03 1.6417814e-03 1.3885655e-03 ... 8.7190222e-04
  1.3210910e-03 6.7584770e-04]]


Inferencing on seed2077 fol

In [16]:
oof_predictions = glob.glob(f'{model_output_folder}/oof_*.npy')[0]
oof_predictions = np.load(oof_predictions)

oof_loss = mean_logloss(oof_predictions, Y)
print(f"OOF Validation Loss: {oof_loss:.6f}")

OOF Validation Loss: 0.015660


# Submission

In [17]:
sub.iloc[:, 1:] = y_pred
# sub.iloc[:, 1:] = np.clip(y_pred, P_MIN, P_MAX)

In [18]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000654,0.000676,0.001698,0.025632,0.024633,0.005388,0.002858,0.005945,0.000319,...,0.000989,0.000928,0.003703,0.001658,0.000592,0.000558,0.001170,0.002343,0.003509,0.001878
1,id_001897cda,0.000346,0.000900,0.001363,0.001496,0.001136,0.001578,0.003759,0.011087,0.037884,...,0.000565,0.001329,0.002702,0.000482,0.009206,0.000676,0.003040,0.000835,0.005522,0.003607
2,id_002429b5b,0.000506,0.000397,0.003634,0.031301,0.029841,0.003032,0.005181,0.004543,0.000495,...,0.001502,0.001087,0.005807,0.004938,0.004451,0.000513,0.002821,0.005713,0.002588,0.004170
3,id_00276f245,0.000678,0.000645,0.002303,0.021629,0.014514,0.004599,0.002557,0.004709,0.000428,...,0.000684,0.001681,0.003239,0.025977,0.011544,0.000641,0.002341,0.002402,0.001936,0.003865
4,id_0027f1083,0.001561,0.001488,0.001885,0.016889,0.021342,0.004013,0.006029,0.001242,0.000358,...,0.000738,0.000730,0.004210,0.002146,0.000871,0.000735,0.001030,0.001664,0.000367,0.001167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.000724,0.000928,0.001337,0.004753,0.007097,0.002214,0.001184,0.005252,0.000818,...,0.000546,0.005248,0.002181,0.230516,0.006775,0.001123,0.002893,0.001384,0.001918,0.001338
3978,id_ff925dd0d,0.005401,0.003200,0.000649,0.006668,0.025671,0.006888,0.005317,0.002813,0.000594,...,0.000395,0.000608,0.002891,0.002626,0.001068,0.001043,0.001474,0.001377,0.000424,0.001075
3979,id_ffb710450,0.001584,0.000791,0.000957,0.011880,0.036116,0.005746,0.003543,0.003405,0.000229,...,0.000438,0.000550,0.001977,0.001781,0.001078,0.000550,0.000667,0.001266,0.000454,0.001111
3980,id_ffbb869f2,0.001581,0.001061,0.001038,0.022086,0.031644,0.006671,0.004764,0.001936,0.000316,...,0.000366,0.000388,0.003078,0.001375,0.001027,0.000484,0.000917,0.001476,0.000459,0.002083


In [19]:
# Set ctl_vehicle to 0
sub.iloc[test_features['cp_type'] == 'ctl_vehicle', 1:] = 0

# Save Submission
# sub.to_csv('submission_2heads-looper-super-puper.csv', index=False)
sub.to_csv('submission.csv', index=False)

In [20]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000654,0.000676,0.001698,0.025632,0.024633,0.005388,0.002858,0.005945,0.000319,...,0.000989,0.000928,0.003703,0.001658,0.000592,0.000558,0.001170,0.002343,0.003509,0.001878
1,id_001897cda,0.000346,0.000900,0.001363,0.001496,0.001136,0.001578,0.003759,0.011087,0.037884,...,0.000565,0.001329,0.002702,0.000482,0.009206,0.000676,0.003040,0.000835,0.005522,0.003607
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000678,0.000645,0.002303,0.021629,0.014514,0.004599,0.002557,0.004709,0.000428,...,0.000684,0.001681,0.003239,0.025977,0.011544,0.000641,0.002341,0.002402,0.001936,0.003865
4,id_0027f1083,0.001561,0.001488,0.001885,0.016889,0.021342,0.004013,0.006029,0.001242,0.000358,...,0.000738,0.000730,0.004210,0.002146,0.000871,0.000735,0.001030,0.001664,0.000367,0.001167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.000724,0.000928,0.001337,0.004753,0.007097,0.002214,0.001184,0.005252,0.000818,...,0.000546,0.005248,0.002181,0.230516,0.006775,0.001123,0.002893,0.001384,0.001918,0.001338
3978,id_ff925dd0d,0.005401,0.003200,0.000649,0.006668,0.025671,0.006888,0.005317,0.002813,0.000594,...,0.000395,0.000608,0.002891,0.002626,0.001068,0.001043,0.001474,0.001377,0.000424,0.001075
3979,id_ffb710450,0.001584,0.000791,0.000957,0.011880,0.036116,0.005746,0.003543,0.003405,0.000229,...,0.000438,0.000550,0.001977,0.001781,0.001078,0.000550,0.000667,0.001266,0.000454,0.001111
3980,id_ffbb869f2,0.001581,0.001061,0.001038,0.022086,0.031644,0.006671,0.004764,0.001936,0.000316,...,0.000366,0.000388,0.003078,0.001375,0.001027,0.000484,0.000917,0.001476,0.000459,0.002083
